In [1]:
# Import the PyTorch library
import torch
import torch.nn as nn
from torch.nn import functional as F

In [13]:
# Set the device to 'mps' if CUDA is available, otherwise set it to 'cpu' for using apple silicon
device = "mps" if torch.backends.mps.is_available() else "cpu"
# Print the device that will be used for computations
print(device)
block_size = 8
batch_size = 4
max_iters = 10000
# eval_interval = 2500
learning_rate = 3e-4
eval_iters = 250
# dropout = 0.2

mps


In [3]:
# Open the file 'wizard_of_oz.txt' in read mode with utf-8 encoding
with open("wizard_of_oz.txt", 'r', encoding='utf-8') as f:
    # Read the entire content of the file into the variable 'text'
    text = f.read()

# Create a sorted set of unique characters present in the 'text'
chars = sorted(set(text))

# Calculate the size of the vocabulary, which is the number of unique characters in the 'text'
vocab_size = len(chars)

In [4]:
# Create a dictionary to map each character to a unique integer
string_to_int = { ch:i for i, ch in enumerate(chars) }

# Create a dictionary to map each integer back to its corresponding character
int_to_string = { i:ch for i, ch in enumerate(chars) }

# Define a function to encode a string into a list of integers
encode = lambda s: [string_to_int[c] for c in s]

# Define a function to decode a list of integers back into a string
decode = lambda l: ''.join([int_to_string[i] for i in l])

# Encode the entire text into a tensor of integers
data = torch.tensor(encode(text), dtype=torch.long)

# Print the first 100 elements of the encoded data
data[:100]

tensor([ 1,  1, 28, 39, 42, 39, 44, 32, 49,  1, 25, 38, 28,  1, 44, 32, 29,  1,
        47, 33, 50, 25, 42, 28,  1, 33, 38,  1, 39, 50,  0,  0,  1,  1, 26, 49,
         0,  0,  1,  1, 36, 11,  1, 30, 42, 25, 38, 35,  1, 26, 25, 45, 37,  0,
         0,  1,  1, 25, 45, 44, 32, 39, 42,  1, 39, 30,  1, 44, 32, 29,  1, 47,
        33, 50, 25, 42, 28,  1, 39, 30,  1, 39, 50,  9,  1, 44, 32, 29,  1, 36,
        25, 38, 28,  1, 39, 30,  1, 39, 50,  9])

In [5]:
# Calculate the number of elements that correspond to 80% of the total data length
n = int(0.8*len(data))

# Split the data into training and validation sets
train_data, val_data = data[:n], data[n:]

def get_batch(split):
    data = train_data if split == "train" else val_data
    ix = torch.randint(len(data) - block_size, (batch_size, ))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y
x, y = get_batch('train')
print('inputs:')
print(x)
print('targets:')
print(y)

inputs:
tensor([[78,  1, 57, 68, 74, 55, 65, 58],
        [ 1,  1,  1,  1,  1,  1,  1,  1],
        [61, 62, 72,  1, 72, 73, 71, 54],
        [ 1, 73, 68, 68, 64,  1, 56, 68]], device='mps:0')
targets:
tensor([[ 1, 57, 68, 74, 55, 65, 58, 23],
        [ 1,  1,  1,  1,  1,  1,  1,  1],
        [62, 72,  1, 72, 73, 71, 54, 67],
        [73, 68, 68, 64,  1, 56, 68, 74]], device='mps:0')


In [6]:
@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

In [7]:
# # Set the block size to 8. This is the number of elements to consider at a time from the training data.
# block_size = 8

# # Split the training data into two parts: 'x' and 'y'. 'x' is the first 'block_size' elements and 'y' is the next 'block_size' elements.
# x, y = train_data[:block_size],  train_data[1:block_size+1]

# # Loop over the range of block_size
# for t in range(block_size):
#     # The context is the first 't+1' elements of 'x'
#     context = x[:t+1]
#     # The target is the 't'-th element of 'y'
#     target = y[t]
#     # Print the context and target
#     print('when input is', context, 'target is', target)

In [8]:
class BigramLanguageModel(nn.Module):

    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

    def forward(self, index, targets=None):
        logits = self.token_embedding_table(index)

        if targets is None:
            loss = None

        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss
    
    def generate(self, index, max_new_tokens):
        # index is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # get the predictions
            logits, loss = self.forward(index)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            index_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            index = torch.cat((index, index_next), dim=1) # (B, T+1)

        return index

In [31]:
import os
model = BigramLanguageModel(vocab_size) if not os.path.exists("wizard_of_oz.pth") else torch.load("wizard_of_oz.pth")
m = model.to(device)

context = torch.zeros((1, 1), dtype=torch.long, device=device)
generated_chars = decode(m.generate(context, max_new_tokens=500)[0].tolist())
print(generated_chars)



"

"OTha gher  wirage goflirobancomava the ine THE unes y Ms t ck nidulon
bou th the wa I'linve Wheck. he towe ce tskith y mardm t?"F?"ALA thy fldotom
 thathen ce budel saresoiggred
"ant bryoundn izal aib.

ather ised ale Sougled ivee bice lline
irm, anthe r  othe verit ad, abowed
"HEuty's. be of

athour



toupithy TAnend war Wiofomel gs  gl anooin'saron.
eythanghebe ots ch be
froo.

ancou win pim la wa  t ensho thexcrathishentocasidr as w th tscas a thy arnd dea S.


ve INluclldllisshenofly a


In [32]:
%%time
# Create a PyTorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for iter in range(max_iters):
    if iter % eval_iters == 0:
        losses = estimate_loss()
        print(f"step: {iter}, train loss: {losses['train']:.3f}, val loss: {losses['val']:.3f}")

    # sample a batch of data
    xb, yb = get_batch('train')
    # evaluate the loss
    logits, loss = model.forward(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

print(loss.item())

step: 0, train loss: 2.433, val loss: 2.487
step: 250, train loss: 2.427, val loss: 2.511
step: 500, train loss: 2.427, val loss: 2.470
step: 750, train loss: 2.425, val loss: 2.480
step: 1000, train loss: 2.428, val loss: 2.493
step: 1250, train loss: 2.420, val loss: 2.493
step: 1500, train loss: 2.430, val loss: 2.476
step: 1750, train loss: 2.416, val loss: 2.494
step: 2000, train loss: 2.434, val loss: 2.488
step: 2250, train loss: 2.437, val loss: 2.479
step: 2500, train loss: 2.440, val loss: 2.460
step: 2750, train loss: 2.423, val loss: 2.486
step: 3000, train loss: 2.428, val loss: 2.477
step: 3250, train loss: 2.436, val loss: 2.467
step: 3500, train loss: 2.391, val loss: 2.483
step: 3750, train loss: 2.417, val loss: 2.493
step: 4000, train loss: 2.422, val loss: 2.486
step: 4250, train loss: 2.432, val loss: 2.494
step: 4500, train loss: 2.441, val loss: 2.457
step: 4750, train loss: 2.426, val loss: 2.459
step: 5000, train loss: 2.437, val loss: 2.488
step: 5250, train l

In [33]:
context = torch.zeros((1, 1), dtype=torch.long, device=device)
generated_chars = decode(m.generate(context, max_new_tokens=500)[0].tolist())
print(generated_chars)


Jik t Wis Pears ttorofupry o the, pe!" bushe lo ney aille Wiourow  ofon a apan s "t ke,"oireat foiale, kid d inlare plastooutheed "QThthedoorpe sasin tot, fr iglawanke gearr.

"
wimaristheal RPro abur a IC5UThe erousare hrt roisscaritod o an fthandif s
sethe thsest theve d ds id,"
wof Bugagas be he w d
ls wowemofaregas


"
sthey. ro owican
The butuputeapliflizand to u outhee y
"thoalm in sasousich can e wando mevend sat Bund.
"TE.
nxilthe cheed ot be al tero
amith b VNCHE tost d ced; sote whes. 


In [30]:
torch.save(m, "wizard_of_oz.pth")